In [ ]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle
from scipy.signal import argrelextrema, argrelmax, argrelmin


import sys, os
sys.path.append(os.path.abspath('..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

plt.rcParams['lines.linewidth'] = 1 

mpl.rcParams.update({'font.size': 10, 'lines.linewidth': 1, 'figure.dpi': 300})


In [ ]:
from minizinc import Instance, Model, Solver, Status as mzStatus
import nest_asyncio
nest_asyncio.apply()

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_blobs
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import ListedColormap

In [ ]:

def call_mzn_model(model_name,model_params):
    with open(f'pars-{model_name}.dzn.json', 'w+') as f: f.write(json.dumps(model_params, indent=2))

    # print('Model params:', model_params)

    mzn_model = Model(f'{model_name}.mzn')
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, mzn_model)

    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()

    return result


In [ ]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = 2, .2
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    return ax

In [ ]:
pair = 'BTC_USDT'
timeframe = '2h'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

In [ ]:
odf['boL'] = odf[['open', 'close']].min(axis=1)
odf['boH'] = odf[['open', 'close']].max(axis=1)
# odf['dirF1'] = odf.cadir.shift(-1).ffill().astype(int)
###
odf['cadir'] = 1 - 2*(odf.close < odf.open).astype(int)
# wdf
mzn_res = call_mzn_model('last-such', {'markers': odf.cadir.values.tolist()})
odf['n_prev'] = mzn_res['prev']

# odf['f_6_close_max'] = odf.close.rolling(6).max().shift(-6)
# roll18 = odf.rolling(18)
# odf['l_18_std']

###
odf.head(5)


In [ ]:
def add_indicators(wdf, lag):
    ###
    laRoll = wdf.rolling(window=lag)
    wdf[f'laMean'] = laRoll.boL.mean()#.bfill()
    wdf[f'laMin'] = laRoll.boL.min()#.bfill()
    wdf[f'laMax'] = laRoll.boL.max()#.bfill()
    wdf[f'laStd'] = laRoll.boL.std() 
    wdf[f'laZScore'] = (wdf.close - wdf.laMean) / wdf.laStd #.bfill()

    laRoll = wdf.rolling(window=lag)
    wdf[f'wd2laMeanP'] = wdf.low.divide(wdf.laMean).apply(np.log) * (wdf.laMean > wdf.low)
    wdf[f'wdSum'] = laRoll.wd2laMeanP.sum()
    wdf[f'wdMin'] = laRoll.wd2laMeanP.min()

    for wlag in [12,24,36,72]:
        wdf[f'c_{wlag}_min'] = wdf.close.rolling(wlag).min()
        wdf[f'c_{wlag}_max'] = wdf.close.rolling(wlag).max()
        wdf[f'l_{wlag}_min'] = wdf.low.rolling(wlag).min()
        wdf[f'h_{wlag}_max'] = wdf.high.rolling(wlag).max()
    # wdf[f'l_18_min'] = wdf.close.rolling(18).min()
    # wdf[f'l_18_max'] = wdf.close.rolling(18).max()
    # wdf[f'l_12_min'] = wdf.close.rolling(12).min()
    # wdf[f'l_12_max'] = wdf.close.rolling(12).max()
    ###
    # for local maxima
    # iUps = argrelextrema(wdf.close.values, np.greater, order=xOrd)
    # iDns = argrelextrema(wdf.close.values, np.less, order=xOrd)
    # ixUps = wdf.index[iUps]
    # ixDns = wdf.index[iDns]
    ###
    return wdf


In [ ]:
wdf = add_indicators(wdf=odf.copy(), lag=4,)
wdf

In [ ]:
wdf

In [ ]:
# import logging

# logging.basicConfig(filename="minizinc-python.log", level=logging.DEBUG)
wdf = None
sig = None
def printer(ws, xOrd, laZScore, par_wick):
    global wdf, sig
    wstart = ws * wlen
    wend = wstart + wlen
    wdf = odf.iloc[wstart:wend].copy()

    wdf = add_indicators(wdf, lag=4)
    sig = ((wdf.laZScore < laZScore) & (wdf.cadir == -1))

    # isig = wdf[sig].index
    
    ###
    fig = plt.figure(figsize=(15,5))
    # fig, (ax1,ax2) = plt.subplots(2,1, height_ratios=[2,1], sharex=True)
    plt.tight_layout()
    ax1,ax2 = fig.subplots(2,1, height_ratios=[2,1], sharex=True)
    wdf.close.plot(ax=ax1, lw=0.5)
    for i in wdf.index[sig]: 
        for ax in [ax1]:
            ax.axvline(i, lw=2, alpha=0.2)
    wdf.wdSum.plot(ax=ax2, secondary_y=True, legend='wdSum')
    wdf.wdMin.plot(ax=ax2, legend=True)
    wdf.laZScore.plot(ax=ax1, secondary_y=True)
    # ax1.set_label(True)
    # ax1.scatter(ixUps, wdf.close[ixUps], c='r', alpha=0.3, s=50)
    # ax1.scatter(ixDns, wdf.close[ixDns], c='g', alpha=0.3, s=50)
    # ax2.plot(wdf.index,wdf.wdSum, c='b')
    plot_candles(wdf,ax=ax1)
    
    # ax1.scatter(ix_dpeaks, wdf.loc[ix_dpeaks].close, c='g', alpha=0.3, s=100)
wlen = 2000
x = interact(printer, 
         ws=IntSlider(min=0, max=odf.shape[0] // wlen, step=1, value=50, description='wn'), 
         xOrd=IntSlider(min=1,max=15,value=6, description='XOrd'),
         laZScore=FloatText(min=0.,max=0.10,step=0.001, value=0.0005, description='laZScore'), 
         par_wick=FloatText(min=0.,max=0.05,step=0.001, description='wick'))

In [ ]:
# wdf.index[sig]
# laRoll

### Naive Bayes

In [ ]:

# wdf

In [ ]:
from sklearn.naive_bayes import GaussianNB
from adspy_shared_utilities import plot_class_regions_for_classifier

adf = wdf[wdf.laZScore >= 0.0005 ].dropna()
X_wdf = adf[['laStd','wdMin']].values
y_wdf = adf.dirF1.values

X_train, X_test, y_train, y_test = train_test_split(X_wdf, y_wdf, random_state=0)

scaler = MinMaxScaler()

X_trainS = scaler.fit_transform(X_train)
X_testS = scaler.transform(X_test)

nbclf = GaussianNB().fit(X_train, y_train)
plot_class_regions_for_classifier(nbclf, X_trainS, y_train, X_testS, y_test,
                                 'Gaussian Naive Bayes classifier: Dataset 1', gkwargs={'figsize':(15,5)})

### Gradient-Boosted Decision Tree

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


adf = wdf[wdf.laZScore >= 0.0005 ].dropna()
X_wdf = adf[['laStd', 'wdSum']].values
y_wdf = adf.dirF1.values

X_train, X_test, y_train, y_test = train_test_split(X_wdf, y_wdf, random_state=0)

# clf = GradientBoostingClassifier(random_state = 0)
# clf.fit(X_train, y_train)

# print('(learning_rate=0.1, max_depth=3)')
# print('Accuracy of GBDT classifier on training set: {:.2f}'
#      .format(clf.score(X_train, y_train)))
# print('Accuracy of GBDT classifier on test set: {:.2f}\n'
#      .format(clf.score(X_test, y_test)))

clf = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 5, random_state = 0)
clf.fit(X_train, y_train)

print('(learning_rate=0.01, max_depth=2)')
print('Accuracy of GBDT classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of GBDT classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

### Neural Nets Classifier

In [ ]:
pair = 'BTC_USDT'
timeframe = '2h'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

In [ ]:
from sklearn.neural_network import MLPClassifier
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot

wdf = wdf.dropna()
X_wdf = wdf[['laStd','wdMin', 'n_prev']].values
y_wdf = wdf.dirF1.values

X_train, X_test, y_train, y_test = train_test_split(X_wdf, y_wdf, random_state=0)

scaler = MinMaxScaler()

X_trainS = scaler.fit_transform(X_train)
X_testS = scaler.transform(X_test)

fig, subaxes = plt.subplots(3, 1, figsize=(15,6))

for units, axis in zip([10, 100], subaxes):
    nnclf = MLPClassifier(hidden_layer_sizes = [units]* 2, solver='lbfgs',
                         random_state = 0).fit(X_trainS, y_train)
    
    title = 'Dataset 1: Neural net classifier, 1 layer, {} units'.format(units)
    
    # plot_class_regions_for_classifier_subplot(nnclf, X_trainS, y_train,
    #                                          X_testS, y_test, title, axis)
    # plt.tight_layout()

In [ ]:
wdf

In [ ]:

df_test_results = pd.DataFrame(columns=['layers', 'units', 'tp', 'fp', 'tn', 'fn', 'precision', 'recall'])
df_test_results.set_index(['layers', 'units'])
df_test_results

In [ ]:

xdf = add_indicators(wdf=odf.copy().dropna(), lag=4)
xdf.columns

In [57]:
from sklearn.neural_network import MLPClassifier
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
import itertools
from tqdm.auto import tqdm

# xdf = wdf.copy()
# y_wdf = wdf.dirF1.values

xdf = add_indicators(wdf=odf.copy().dropna(), lag=4)
fu_N = 6
xdf['fu_top'] = wdf.close.rolling(fu_N).max().shift(-fu_N).divide(xdf.close).apply(np.log)
xdf['fu_bottom'] = wdf.low.rolling(fu_N).min().shift(-fu_N).divide(xdf.close).apply(np.log)

xdf = xdf.dropna()

perc = 0.02
r2r = 2
y_xdf = ((xdf.fu_top >= perc) & (xdf.fu_bottom > - perc / r2r)
         )


X_xdf = xdf[['open', 'high', 'low', 'close', 'volume', 'boL', 'boH',
       'cadir', 'n_prev', 'laMean', 'laMin', 'laMax', 'laStd', 'laZScore',
       'wd2laMeanP', 'wdSum', 'wdMin', 'c_12_min', 'c_12_max', 'l_12_min',
       'h_12_max', 'c_24_min', 'c_24_max', 'l_24_min', 'h_24_max', 'c_36_min',
       'c_36_max', 'l_36_min', 'h_36_max', 'c_72_min', 'c_72_max', 'l_72_min',
       'h_72_max']].values

X_train, X_test, y_train, y_test = train_test_split(X_xdf, y_xdf, random_state=1)

scaler = MinMaxScaler()

X_trainS = scaler.fit_transform(X_train)
X_testS = scaler.transform(X_test)

# df_test_results = pd.DataFrame(columns=['layers', 'units', 'tp', 'fp', 'tn', 'fn', 'precision', 'recall'])
# df_train_results = pd.DataFrame(columns=['layers', 'units', 'tp', 'fp', 'tn', 'fn', 'precision', 'recall'])
test_scores_rows = []
train_conf_mat = None
# ,60,80,100,110, 120
for layers,units in tqdm(itertools.product([2,3,4,5,4,7,8,9,10], [10,15,20,30,40,50,60,80])):
    clf = MLPClassifier(hidden_layer_sizes = [units]* layers, solver='adam',
                            learning_rate='adaptive',
                            random_state = 0, max_iter=200
                         ).fit(X_trainS, y_train)
    
    train_score = clf.score(X_trainS, y_train)
    test_score  = clf.score(X_testS, y_test)
    print('================================================================')
    print(f'Neural net classifier, {layers} layer, {units} units')
    print(f"Train score = {train_score:.2f}, Test score = {test_score:.2f}")
    print(f'---------------------------------------------------------------')

    y_train_pred = clf.predict(X_trainS)
    train_conf_mat = confusion_matrix(y_train, y_train_pred)
    print(f'Training confusion_matrix: \n{train_conf_mat}')
    # train_precision = precision_score(y_train, y_train_pred)
    # train_recall = recall_score(y_train, y_train_pred)


    # print(f'Train precision = {round(train_precision,2)}, recall = {round(train_recall,2)}')
     
    print(f'----------')
    y_test_pred = clf.predict(X_testS)
    test_conf_mat = confusion_matrix(y_test, y_test_pred)
    print(f'Testing confusion matrix: \n{test_conf_mat}')
    # test_precision = precision_score(y_test, y_test_pred)
    # test_recall = recall_score(y_test, y_test_pred)
    # print(f'Test precision = {round(test_precision,2)}, recall = {round(test_recall,2)}')

    # df_train_results.append({'layers': layers, 'units': units, 'tp': tr_tp, 'fn': tr_fn, 'tn': tr_tn, 'fp': tr_fp})
    # test_scores_rows.append({'layers': layers, 'units': units, 'tp': tst_tp, 'fn': tst_fn, 'tn': tst_tn, 'fp': tst_fp})

    test_scores_rows.append({'layers': layers, 'units': units, 'test_conf_mat': test_conf_mat, 'train_conf_mat': train_conf_mat})


    # df_test_scores.loc([layers, units]) = {'tp': tst_tp, 'fn': tst_fn, 'tn': tst_tn, 'fp': tst_fp})

#     break
    # plot_class_regions_for_classifier_subplot(nnclf, X_trainS, y_train,
                                            #  X_testS, y_test, title, axis)
    # plt.tight_layout()

Neural net classifier, 2 layer, 80 units
Train score = 0.87, Test score = 0.89
---------------------------------------------------------------
Training confusion_matrix: 
[[349   2]
 [ 51  12]]
----------
Testing confusion matrix: 
[[118   0]
 [ 15   5]]
Neural net classifier, 3 layer, 10 units
Train score = 0.85, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[351   0]
 [ 63   0]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]
Neural net classifier, 3 layer, 15 units
Train score = 0.85, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[351   0]
 [ 63   0]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]
Neural net classifier, 3 layer, 20 units
Train score = 0.85, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[351   0]
 [ 63   0]]
----------
Testing confusion matrix: 
[[1

/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural net classifier, 3 layer, 30 units
Train score = 0.86, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[350   1]
 [ 57   6]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]


/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural net classifier, 3 layer, 40 units
Train score = 0.87, Test score = 0.88
---------------------------------------------------------------
Training confusion_matrix: 
[[343   8]
 [ 46  17]]
----------
Testing confusion matrix: 
[[115   3]
 [ 14   6]]
Neural net classifier, 3 layer, 50 units
Train score = 0.86, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[351   0]
 [ 56   7]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]


...

Neural net classifier, 4 layer, 40 units
Train score = 0.87, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[346   5]
 [ 48  15]]
----------
Testing confusion matrix: 
[[116   2]
 [ 17   3]]
Neural net classifier, 4 layer, 50 units
Train score = 0.88, Test score = 0.88
---------------------------------------------------------------
Training confusion_matrix: 
[[350   1]
 [ 50  13]]
----------
Testing confusion matrix: 
[[118   0]
 [ 16   4]]
Neural net classifier, 4 layer, 60 units
Train score = 0.87, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[350   1]
 [ 51  12]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]
Neural net classifier, 4 layer, 80 units
Train score = 0.86, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[350   1]
 [ 55   8]]
----------
Testing confusion matrix: 
[[1

/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural net classifier, 7 layer, 15 units
Train score = 0.87, Test score = 0.86
---------------------------------------------------------------
Training confusion_matrix: 
[[350   1]
 [ 54   9]]
----------
Testing confusion matrix: 
[[118   0]
 [ 20   0]]


/home/grenada/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural net classifier, 7 layer, 20 units
Train score = 0.87, Test score = 0.87
---------------------------------------------------------------
Training confusion_matrix: 
[[342   9]
 [ 45  18]]
----------
Testing confusion matrix: 
[[114   4]
 [ 14   6]]
Neural net classifier, 7 layer, 30 units
Train score = 0.86, Test score = 0.87
---------------------------------------------------------------
Training confusion_matrix: 
[[349   2]
 [ 56   7]]
----------
Testing confusion matrix: 
[[118   0]
 [ 18   2]]


In [ ]:
# np.asmatrix(df_test_scores.test_conf_mat.apply(lambda x: x.ravel().tolist()).values.tolist()).T

In [ ]:
# test_scores_rows
df_test_scores = pd.DataFrame.from_dict(test_scores_rows)
# df_test_scores['precision'] = df_test_scores.tp / (df_test_scores.tp + )
# df_test_scores['test_precision'] = df_test_scores.test_conf_mat.apply(lambda x: round(1. / (1e-08 + x[1,1] + x[0,1]), 2) if (x[1,1] + x[0,1]) > 0 else 0)

# df_test_scores.loc[:]['TN', 'FP', 'FN', 'TP'] = np.asmatrix(df_test_scores.test_conf_mat.apply(lambda x: x.ravel().tolist()).values.tolist())
df_test_scores = df_test_scores.join(pd.DataFrame(np.asmatrix(df_test_scores.test_conf_mat.apply(lambda x: x.ravel().tolist()).values.tolist()), 
             columns=['TN', 'FP', 'FN', 'TP']))

df_test_scores['precision'] = (df_test_scores.TP / (df_test_scores.TP + df_test_scores.FP + 1e-08)).round(2)
df_test_scores['recall'] = (df_test_scores.TP / (df_test_scores.TP + df_test_scores.FN + 1e-08)).round(2)

df_test_scores.head(20)#[df_test_scores.test_precision > 0]

In [ ]:
df_test_scores[df_test_scores.precision > 0.4].sort_values(['precision', 'recall'])
# df_test_scores[df_test_scores.test_conf_mat[1,1] > 0].head(20)

In [ ]:
df_test_scores.test_conf_mat.apply(lambda x: x.ravel()).values

In [ ]:
wdf

In [ ]:
odf